<a href="https://colab.research.google.com/github/j-hossain/BlogSite/blob/main/Copy_of_Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# first upload the kaggle api key to fetch dataset
from google.colab import files
apiKey = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
! chmod 600 /content/kaggle.json
! kaggle datasets download solarmainframe/ids-intrusion-csv
! mkdir dataset
! unzip /content/ids-intrusion-csv.zip -d '/content/dataset'

100% 1.59G/1.60G [00:19<00:00, 96.0MB/s]
100% 1.60G/1.60G [00:19<00:00, 86.3MB/s]
Archive:  /content/ids-intrusion-csv.zip
  inflating: /content/dataset/02-14-2018.csv  
  inflating: /content/dataset/02-15-2018.csv  
  inflating: /content/dataset/02-16-2018.csv  
  inflating: /content/dataset/02-20-2018.csv  
  inflating: /content/dataset/02-21-2018.csv  
  inflating: /content/dataset/02-22-2018.csv  
  inflating: /content/dataset/02-23-2018.csv  
  inflating: /content/dataset/02-28-2018.csv  
  inflating: /content/dataset/03-01-2018.csv  
  inflating: /content/dataset/03-02-2018.csv  


In [29]:
dtp = {}
drp = ["Timestamp"]

In [30]:
df=pd.read_csv('/content/dataset/02-14-2018.csv',dtype=dtp,usecols=lambda x: x not in drp)

In [31]:
for i, name in enumerate(df.columns):
  if name=='Label':
    pass
  elif df[name].dtype=="int64":
    if df[name].min()>=0:
      if df[name].max()<=255:
        dtp[name]="uint8"
      elif df[name].max()<=65535:
        dtp[name]="uint16"
      elif df[name].max()<=2147483647:
        dtp[name]="uint32"
    else:
      if df[name].min()>=-128 and df[name].max()<=127:
        dtp[name]="int8"
      elif df[name].min()>=-32768 and df[name].max()<=32767:
        dtp[name]="int16"
      elif df[name].min()>=-2147483648 and df[name].max()<=2147483647:
        dtp[name]="int32"
  else:
      if df[name].min()>=-2147483648 and df[name].max()<=2147483647:
        dtp[name]="float32"
  
print(dtp)

{'Dst Port': 'uint16', 'Protocol': 'uint8', 'Tot Fwd Pkts': 'uint16', 'Tot Bwd Pkts': 'uint16', 'TotLen Fwd Pkts': 'uint32', 'TotLen Bwd Pkts': 'uint32', 'Fwd Pkt Len Max': 'uint16', 'Fwd Pkt Len Min': 'uint16', 'Fwd Pkt Len Mean': 'float32', 'Fwd Pkt Len Std': 'float32', 'Bwd Pkt Len Max': 'uint16', 'Bwd Pkt Len Min': 'uint16', 'Bwd Pkt Len Mean': 'float32', 'Bwd Pkt Len Std': 'float32', 'Bwd IAT Tot': 'uint32', 'Bwd IAT Mean': 'float32', 'Bwd IAT Std': 'float32', 'Bwd IAT Max': 'uint32', 'Bwd IAT Min': 'uint32', 'Fwd PSH Flags': 'uint8', 'Bwd PSH Flags': 'uint8', 'Fwd URG Flags': 'uint8', 'Bwd URG Flags': 'uint8', 'Fwd Header Len': 'uint32', 'Bwd Header Len': 'uint32', 'Fwd Pkts/s': 'float32', 'Bwd Pkts/s': 'float32', 'Pkt Len Min': 'uint16', 'Pkt Len Max': 'uint16', 'Pkt Len Mean': 'float32', 'Pkt Len Std': 'float32', 'Pkt Len Var': 'float32', 'FIN Flag Cnt': 'uint8', 'SYN Flag Cnt': 'uint8', 'RST Flag Cnt': 'uint8', 'PSH Flag Cnt': 'uint8', 'ACK Flag Cnt': 'uint8', 'URG Flag Cnt': 

In [32]:
df=pd.read_csv('/content/dataset/02-14-2018.csv',dtype=dtp,usecols=lambda x: x not in drp)

In [33]:
pd.set_option('display.max_rows', 500)
for i, name in enumerate(df.columns):
    # print(f'"{name}":{{"dtype":{df[name].dtype}, "min":{df[name].min()}, "max":{df[name].max()}}} {"," if i>(len(df.columns)-1) else ""}')
    print(f'"{name}":{{"dtype":{df[name].dtype}, "min":{len(df[name].unique())}}} {"," if i>(len(df.columns)-1) else ""}')
print("}") 

"Dst Port":{"dtype":uint16, "min":18567} 
"Protocol":{"dtype":uint8, "min":3} 
"Flow Duration":{"dtype":int64, "min":389493} 
"Tot Fwd Pkts":{"dtype":uint16, "min":751} 
"Tot Bwd Pkts":{"dtype":uint16, "min":988} 
"TotLen Fwd Pkts":{"dtype":uint32, "min":5842} 
"TotLen Bwd Pkts":{"dtype":uint32, "min":15656} 
"Fwd Pkt Len Max":{"dtype":uint16, "min":1390} 
"Fwd Pkt Len Min":{"dtype":uint16, "min":112} 
"Fwd Pkt Len Mean":{"dtype":float32, "min":17879} 
"Fwd Pkt Len Std":{"dtype":float32, "min":27491} 
"Bwd Pkt Len Max":{"dtype":uint16, "min":958} 
"Bwd Pkt Len Min":{"dtype":uint16, "min":270} 
"Bwd Pkt Len Mean":{"dtype":float32, "min":24517} 
"Bwd Pkt Len Std":{"dtype":float32, "min":28813} 
"Flow Byts/s":{"dtype":float64, "min":454333} 
"Flow Pkts/s":{"dtype":float64, "min":417167} 
"Flow IAT Mean":{"dtype":float64, "min":418622} 
"Flow IAT Std":{"dtype":float64, "min":416669} 
"Flow IAT Max":{"dtype":int64, "min":295378} 
"Flow IAT Min":{"dtype":int64, "min":66507} 
"Fwd IAT Tot":{"

In [34]:
d2 = pd.read_csv('/content/dataset/02-15-2018.csv',dtype=dtp,usecols=lambda x: x not in drp)

In [35]:
d3 = pd.read_csv('/content/dataset/02-16-2018.csv',usecols=lambda x: x not in drp)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [38]:
# Some columns have mixed values like 1,2,'3',4,'2' etc.
# taking them to numric while handling the null values in errors
for i, name in enumerate(d3.columns):
  if name=='Label':
    pass
  elif d3[name].dtype==object:
    d3[name] = pd.to_numeric(d3[name], errors='coerce').convert_dtypes()


In [40]:
d3['Label'].unique()

array(['Benign', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk', 'Label'],
      dtype=object)

In [41]:
d4= d3.dropna()

In [42]:
d3=d4

In [43]:
for i, name in enumerate(d3.columns):
  if name=='Label':
    pass
  elif d3[name].dtype=="Int64":
    if d3[name].min()>=0:
      if d3[name].max()<=255:
        d3[name]= d3[name].astype("uint8")
      elif d3[name].max()<=65535:
        d3[name]= d3[name].astype("uint16")
      elif d3[name].max()<=2147483647:
        d3[name]= d3[name].astype("uint32")
    else:
      if d3[name].min()>=-128 and d3[name].max()<=127:
        d3[name]= d3[name].astype("int8")
      elif d3[name].min()>=-32768 and d3[name].max()<=32767:
        d3[name]= d3[name].astype("int16")
      elif d3[name].min()>=-2147483648 and d3[name].max()<=2147483647:
        d3[name]= d3[name].astype("int32")
  else:
      if d3[name].min()>=-2147483648 and d3[name].max()<=2147483647:
        d3[name]= d3[name].astype("float32")

In [44]:
d3['Label'].unique()

array(['Benign', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk'],
      dtype=object)

In [45]:
d2['Label'].unique()

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris'],
      dtype=object)

In [46]:
d4 = d2.dropna()

In [47]:
d2 = d4

In [48]:
df = pd.concat([d2.reset_index(drop=True),d3.reset_index(drop=True)],axis=0)

In [49]:
df['Label'].unique()

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris',
       'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk'], dtype=object)

In [50]:
df.isna().sum().sum()

0

In [51]:
# taking day9 in the variable d2
d2 = pd.read_csv('/content/dataset/03-01-2018.csv',usecols=lambda x: x not in drp)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
pd.set_option('display.max_rows', 500)
for i, name in enumerate(d2.columns):
    # print(f'"{name}":{{"dtype":{df[name].dtype}, "min":{df[name].min()}, "max":{df[name].max()}}} {"," if i>(len(df.columns)-1) else ""}')
    print(f'"{name}":{{"dtype":{d2[name].dtype}, "min":{len(d2[name].unique())}}} {"," if i>(len(d2.columns)-1) else ""}')
print("}") 

"Dst Port":{"dtype":object, "min":20849} 
"Protocol":{"dtype":object, "min":7} 
"Flow Duration":{"dtype":object, "min":159256} 
"Tot Fwd Pkts":{"dtype":object, "min":445} 
"Tot Bwd Pkts":{"dtype":object, "min":573} 
"TotLen Fwd Pkts":{"dtype":object, "min":5599} 
"TotLen Bwd Pkts":{"dtype":object, "min":10875} 
"Fwd Pkt Len Max":{"dtype":object, "min":2217} 
"Fwd Pkt Len Min":{"dtype":object, "min":200} 
"Fwd Pkt Len Mean":{"dtype":object, "min":12241} 
"Fwd Pkt Len Std":{"dtype":object, "min":15816} 
"Bwd Pkt Len Max":{"dtype":object, "min":1270} 
"Bwd Pkt Len Min":{"dtype":object, "min":493} 
"Bwd Pkt Len Mean":{"dtype":object, "min":15118} 
"Bwd Pkt Len Std":{"dtype":object, "min":16435} 
"Flow Byts/s":{"dtype":object, "min":177797} 
"Flow Pkts/s":{"dtype":object, "min":161005} 
"Flow IAT Mean":{"dtype":object, "min":151550} 
"Flow IAT Std":{"dtype":object, "min":129311} 
"Flow IAT Max":{"dtype":object, "min":90505} 
"Flow IAT Min":{"dtype":object, "min":27932} 
"Fwd IAT Tot":{"dtyp

It also has mixed values

In [53]:
d2['Label'].unique()

array(['Benign', 'Label', 'Infilteration'], dtype=object)

In [54]:
d2 = d2.dropna()

In [55]:
for i, name in enumerate(d2.columns):
  if name=='Label':
    pass
  elif d2[name].dtype==object:
    d2[name] = pd.to_numeric(d2[name], errors='coerce').convert_dtypes()


In [56]:
d2 = d2.dropna()

In [57]:
for i, name in enumerate(d2.columns):
  if name=='Label':
    pass
  elif d2[name].dtype=="Int64":
    if d2[name].min()>=0:
      if d2[name].max()<=255:
        d2[name]= d2[name].astype("uint8")
      elif d2[name].max()<=65535:
        d2[name]= d2[name].astype("uint16")
      elif d2[name].max()<=2147483647:
        d2[name]= d2[name].astype("uint32")
    else:
      if d2[name].min()>=-128 and d2[name].max()<=127:
        d2[name]= d2[name].astype("int8")
      elif d2[name].min()>=-32768 and d2[name].max()<=32767:
        d2[name]= d2[name].astype("int16")
      elif d2[name].min()>=-2147483648 and d2[name].max()<=2147483647:
        d2[name]= d2[name].astype("int32")
  else:
      if d2[name].min()>=-2147483648 and d2[name].max()<=2147483647:
        d2[name]= d2[name].astype("float32")

In [58]:
pd.set_option('display.max_rows', 500)
for i, name in enumerate(d2.columns):
    # print(f'"{name}":{{"dtype":{df[name].dtype}, "min":{df[name].min()}, "max":{df[name].max()}}} {"," if i>(len(df.columns)-1) else ""}')
    print(f'"{name}":{{"dtype":{d2[name].dtype}, "min":{len(d2[name].unique())}}} {"," if i>(len(d2.columns)-1) else ""}')
print("}") 

"Dst Port":{"dtype":uint16, "min":16134} 
"Protocol":{"dtype":uint8, "min":3} 
"Flow Duration":{"dtype":uint32, "min":153887} 
"Tot Fwd Pkts":{"dtype":uint16, "min":328} 
"Tot Bwd Pkts":{"dtype":uint16, "min":408} 
"TotLen Fwd Pkts":{"dtype":uint32, "min":3953} 
"TotLen Bwd Pkts":{"dtype":uint32, "min":8805} 
"Fwd Pkt Len Max":{"dtype":uint16, "min":1340} 
"Fwd Pkt Len Min":{"dtype":uint16, "min":126} 
"Fwd Pkt Len Mean":{"dtype":float32, "min":10070} 
"Fwd Pkt Len Std":{"dtype":float32, "min":13855} 
"Bwd Pkt Len Max":{"dtype":uint16, "min":857} 
"Bwd Pkt Len Min":{"dtype":uint16, "min":273} 
"Bwd Pkt Len Mean":{"dtype":float32, "min":13136} 
"Bwd Pkt Len Std":{"dtype":float32, "min":14788} 
"Flow Byts/s":{"dtype":Float64, "min":171540} 
"Flow Pkts/s":{"dtype":Float64, "min":155521} 
"Flow IAT Mean":{"dtype":float32, "min":146237} 
"Flow IAT Std":{"dtype":float32, "min":125735} 
"Flow IAT Max":{"dtype":uint32, "min":83986} 
"Flow IAT Min":{"dtype":uint32, "min":24730} 
"Fwd IAT Tot":{

In [59]:
df = df = pd.concat([df.reset_index(drop=True),d2.reset_index(drop=True)],axis=0)

In [60]:
df['Label'].unique()

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris',
       'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk', 'Infilteration'],
      dtype=object)

In [62]:
df.isna().sum().sum()

0

In [69]:
# taking day10 in the variable d2
d2 = pd.read_csv('/content/dataset/03-02-2018.csv',usecols=lambda x: x not in drp)

In [70]:
d2.Label.unique()

array(['Benign', 'Bot'], dtype=object)

In [71]:
d2 = d2.dropna()

In [72]:
for i, name in enumerate(d2.columns):
  if name=='Label':
    pass
  elif d2[name].dtype==object:
    d2[name] = pd.to_numeric(d2[name], errors='coerce').convert_dtypes()


In [73]:
pd.set_option('display.max_rows', 500)
for i, name in enumerate(d2.columns):
    # print(f'"{name}":{{"dtype":{df[name].dtype}, "min":{df[name].min()}, "max":{df[name].max()}}} {"," if i>(len(df.columns)-1) else ""}')
    print(f'"{name}":{{"dtype":{d2[name].dtype}, "min":{len(d2[name].unique())}}} {"," if i>(len(d2.columns)-1) else ""}')
print("}") 

"Dst Port":{"dtype":int64, "min":37284} 
"Protocol":{"dtype":int64, "min":3} 
"Flow Duration":{"dtype":int64, "min":442344} 
"Tot Fwd Pkts":{"dtype":int64, "min":754} 
"Tot Bwd Pkts":{"dtype":int64, "min":1064} 
"TotLen Fwd Pkts":{"dtype":int64, "min":6266} 
"TotLen Bwd Pkts":{"dtype":float64, "min":18066} 
"Fwd Pkt Len Max":{"dtype":int64, "min":1424} 
"Fwd Pkt Len Min":{"dtype":int64, "min":184} 
"Fwd Pkt Len Mean":{"dtype":float64, "min":20086} 
"Fwd Pkt Len Std":{"dtype":float64, "min":33062} 
"Bwd Pkt Len Max":{"dtype":int64, "min":993} 
"Bwd Pkt Len Min":{"dtype":int64, "min":287} 
"Bwd Pkt Len Mean":{"dtype":float64, "min":29443} 
"Bwd Pkt Len Std":{"dtype":float64, "min":34946} 
"Flow Byts/s":{"dtype":float64, "min":458104} 
"Flow Pkts/s":{"dtype":float64, "min":455755} 
"Flow IAT Mean":{"dtype":float64, "min":400220} 
"Flow IAT Std":{"dtype":float64, "min":524747} 
"Flow IAT Max":{"dtype":float64, "min":195722} 
"Flow IAT Min":{"dtype":float64, "min":44058} 
"Fwd IAT Tot":{"dt

In [74]:
d2 = d2.dropna()

In [75]:
for i, name in enumerate(d2.columns):
  if name=='Label':
    pass
  elif d2[name].dtype=="int64":
    if d2[name].min()>=0:
      if d2[name].max()<=255:
        d2[name]= d2[name].astype("uint8")
      elif d2[name].max()<=65535:
        d2[name]= d2[name].astype("uint16")
      elif d2[name].max()<=2147483647:
        d2[name]= d2[name].astype("uint32")
    else:
      if d2[name].min()>=-128 and d2[name].max()<=127:
        d2[name]= d2[name].astype("int8")
      elif d2[name].min()>=-32768 and d2[name].max()<=32767:
        d2[name]= d2[name].astype("int16")
      elif d2[name].min()>=-2147483648 and d2[name].max()<=2147483647:
        d2[name]= d2[name].astype("int32")
  else:
      if d2[name].min()>=-2147483648 and d2[name].max()<=2147483647:
        d2[name]= d2[name].astype("float32")

In [76]:
pd.set_option('display.max_rows', 500)
for i, name in enumerate(d2.columns):
    # print(f'"{name}":{{"dtype":{df[name].dtype}, "min":{df[name].min()}, "max":{df[name].max()}}} {"," if i>(len(df.columns)-1) else ""}')
    print(f'"{name}":{{"dtype":{d2[name].dtype}, "min":{len(d2[name].unique())}}} {"," if i>(len(d2.columns)-1) else ""}')
print("}") 

"Dst Port":{"dtype":uint16, "min":37284} 
"Protocol":{"dtype":uint8, "min":3} 
"Flow Duration":{"dtype":uint32, "min":442344} 
"Tot Fwd Pkts":{"dtype":uint16, "min":754} 
"Tot Bwd Pkts":{"dtype":uint32, "min":1064} 
"TotLen Fwd Pkts":{"dtype":uint32, "min":6266} 
"TotLen Bwd Pkts":{"dtype":float32, "min":18066} 
"Fwd Pkt Len Max":{"dtype":uint16, "min":1424} 
"Fwd Pkt Len Min":{"dtype":uint16, "min":184} 
"Fwd Pkt Len Mean":{"dtype":float32, "min":20086} 
"Fwd Pkt Len Std":{"dtype":float32, "min":33049} 
"Bwd Pkt Len Max":{"dtype":uint16, "min":993} 
"Bwd Pkt Len Min":{"dtype":uint16, "min":287} 
"Bwd Pkt Len Mean":{"dtype":float32, "min":29440} 
"Bwd Pkt Len Std":{"dtype":float32, "min":34882} 
"Flow Byts/s":{"dtype":float64, "min":458104} 
"Flow Pkts/s":{"dtype":float64, "min":455755} 
"Flow IAT Mean":{"dtype":float32, "min":399468} 
"Flow IAT Std":{"dtype":float32, "min":510417} 
"Flow IAT Max":{"dtype":float32, "min":195722} 
"Flow IAT Min":{"dtype":float32, "min":44058} 
"Fwd IAT 

In [77]:
df = df = pd.concat([df.reset_index(drop=True),d2.reset_index(drop=True)],axis=0)

In [78]:
df.Label.unique()

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris',
       'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk', 'Infilteration',
       'Bot'], dtype=object)

In [79]:
columns_to_drop = [
    'Dst Port',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Fwd URG Flags',
    'Bwd URG Flags',
    'Flow Byts/s',  # This field had np.inf values during training, as such was removed
    'Flow Pkts/s'  # This field had np.inf values during training, as such was removed
]

In [80]:
df.drop(columns=columns_to_drop, inplace=True)

In [81]:
df.shape

(3467511, 72)

In [82]:
df.drop_duplicates(inplace=True)

In [83]:
df.shape

(2335597, 72)

In [84]:
df.Label.unique()

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris',
       'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk', 'Infilteration',
       'Bot'], dtype=object)

Now creating the modified datasets

In [103]:
d0 = df.replace('Benign', 0) # here converting the categorical values to numeric
d1 = d0.replace('Bot', 1)
d2 = d1.replace('DoS attacks-GoldenEye', 2)
d3 = d2.replace('DoS attacks-Hulk', 3)
d4 = d3.replace('DoS attacks-SlowHTTPTest', 4)
d5 = d4.replace('DoS attacks-Slowloris', 5)
d6 = d5.replace('Infilteration', 6)
df = d6
d0=d1=d2=d3=d4=d5=d6=''

In [111]:
df = df.astype({"Protocol": str})

In [112]:
df.Protocol.unique()

array(['0', '6', '17'], dtype=object)

In [114]:
df = pd.get_dummies(df, columns=['Protocol'], drop_first=True)

In [115]:
# For reproducible results
RANDOM_STATE_SEED = 420
train, test = train_test_split(df, test_size=0.3, random_state=RANDOM_STATE_SEED)

In [116]:
train

Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
160215         2165024             8             7             1128   
734136             575             3             4              161   
1504727          34965             2             0                0   
645754            1244             1             1               37   
151970           11516             3             4              326   
...                ...           ...           ...              ...   
10394          5071749             4             4              286   
973237         1594063             8             7             1138   
37908           131557             2             0                0   
198376         1702114             8             7             1144   
1323212          43800             3             0                0   

         TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
160215            1581.0              661                0        141.000000   
734136             488.0              161                0         53.666668   
1504727              0.0                0                0          0.000000   
645754              77.0               37               37         37.000000   
151970             129.0              326                0        108.666664   
...                  ...              ...              ...               ...   
10394              972.0              286                0         71.500000   
973237            1581.0              677                0        142.250000   
37908                0.0                0                0          0.000000   
198376            1581.0              677                0        143.000000   
1323212              0.0                0                0          0.000000   

         Fwd Pkt Len Std  Bwd Pkt Len Max  ...  Active Std  Active Max  \
160215        222.623322             1173  ...         0.0         0.0   
734136         92.953392              488  ...         0.0         0.0   
1504727         0.000000                0  ...         0.0         0.0   
645754          0.000000               77  ...         0.0         0.0   
151970        188.216187              112  ...         0.0         0.0   
...                  ...              ...  ...         ...         ...   
10394         143.000000              972  ...         0.0         0.0   
973237        227.746567             1173  ...         0.0         0.0   
37908           0.000000                0  ...         0.0         0.0   
198376        227.969925             1173  ...         0.0         0.0   
1323212         0.000000                0  ...         0.0         0.0   

         Active Min  Idle Mean  Idle Std  Idle Max  Idle Min  Label  \
160215          0.0        0.0       0.0       0.0       0.0      0   
734136          0.0        0.0       0.0       0.0       0.0      0   
1504727         0.0        0.0       0.0       0.0       0.0      3   
645754          0.0        0.0       0.0       0.0       0.0      0   
151970          0.0        0.0       0.0       0.0       0.0      1   
...             ...        ...       ...       ...       ...    ...   
10394           0.0        0.0       0.0       0.0       0.0      2   
973237          0.0        0.0       0.0       0.0       0.0      0   
37908           0.0        0.0       0.0       0.0       0.0      0   
198376          0.0        0.0       0.0       0.0       0.0      0   
1323212         0.0        0.0       0.0       0.0       0.0      3   

         Protocol_17  Protocol_6  
160215             0           1  
734136             0           1  
1504727            0           1  
645754             1           0  
151970             0           1  
...              ...         ...  
10394              0           1  
973237             0           1  
37908              0           0  
198376             0           1  
1323212            0           1  

[1634917 rows x 73 columns]

In [117]:
numerical_cols = df.columns[:-3]
numerical_cols

Index(['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Header Len', 'Bwd Header Len',
       'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max',
       'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt',
       'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt',
       'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio',
       'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'B

In [121]:
min_max_scaler = MinMaxScaler().fit(train[numerical_cols])

In [122]:
train[numerical_cols] = min_max_scaler.transform(train[numerical_cols])

In [123]:
test[numerical_cols] = min_max_scaler.transform(test[numerical_cols])

In [124]:
y_train = np.array(train.pop("Label"))
X_train = train.values

In [125]:
y_test = np.array(test.pop("Label"))
X_test = test.values

In [126]:
#create the model with 100 trees
model = KNeighborsClassifier(n_neighbors=3)
model = model.fit(X_train, y_train)

In [ ]:
predicted = model.predict(X_test)